In [ ]:
#Testing for flowers
input_path = 'datasetflowers/flowers-102/jpg'   
output_path = 'datasetflowers/flowers-102/resized'

#Testing for celebA

# input_path = "datasetCeleb/celeba/jpg"   
# output_path = "datasetCeleb/celeba/resized"

In [ ]:
!pip install pytorch-fid

In [ ]:
import torchvision
from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms, datasets
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
import os

def resize_images(input_folder, output_folder, size=(500, 500)):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            # Open the image
            img_path = os.path.join(input_folder, filename)
            img = Image.open(img_path)

            # Resize the image
            resized_img = img.resize(size)

            # Save the resized image to the output folder
            output_path = os.path.join(output_folder, filename)
            resized_img.save(output_path)




resize_images(input_path, output_path)

In [ ]:
!python -m pytorch_fid datasetflowers/flowers-102/resized results --dims 64